# Step 5. Flower Image Recommender - Convolutional Neural Network and Cosine Similarity

To recommend similar images from a reference image input from the user, we are using the following approach:

1. Classify the reference image: Using our developed classifier model.
2. Generate feature vector using a Convolutional Neural Network
3. Use k-Nearest Neighbor machine learning algorithm to find the similar images and recommend them.